# **Projeto LLM - Modelo Gemma**

In [ ]:
# Instalando pacotes
#!pip install -U keras-nlp
#!pip install -U keras
#!pip install gemma

In [ ]:
# Instalando pacote
#!pip install huggingface-hub

In [ ]:
# Importando bibliotecas

# bibliotecas sistema
import re
import string
import collections
from collections import Counter

# Biblioteca para manipulação dados
import pandas as pd
import numpy as np

# Biblioteca visualização de dados
import matplotlib.pyplot as plt
import seaborn as sns

# Biblioteca NLP
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Biblioteca mensagens de alertas
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Versão keras tensorflow
import keras
import tensorflow as tf

print("Versão tensorflow:", tf.__version__)
print("Versão keras:", keras.__version__)

In [ ]:
# Versão GPU
!nvcc --version

In [ ]:
# Tipo GPU
!nvidia-smi

In [ ]:
# Verifique se a GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Dispositivo disponível:", device)

In [ ]:
# Base dados
df = pd.read_csv('/content/Tweets.csv')
df

In [ ]:
# Visualizando 5 primeiros dados
df.head()

In [ ]:
# Visualizando 5 últimos dados
df.tail()

In [ ]:
# Visualziando linhas e colunas
df.shape

## **Pré-processamento**

In [ ]:
# Info dados
df.info()

In [ ]:
# Tipo de dados
df.dtypes

In [ ]:
# Converter todos os valores da coluna "clean_text" para strings
df['text'] = df['text'].astype(str)

In [ ]:
# Converter todos os valores da coluna "clean_text" para strings
df['selected_text'] = df['selected_text'].astype(str)

In [ ]:
# Converter todos os valores da coluna "clean_text" para strings
df['sentiment'] = df['sentiment'].astype(str)

In [ ]:
# Excluir coluna
df = df.drop(['textID'], axis=1)
df.head()

In [ ]:
# Função para pré-processamento do texto
def preprocess_text(text):
    if isinstance(text, str):

        # Verificar se o texto é uma string
        # Converter texto para minúsculas
        text = text.lower()

        # Remover emojis
        emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # símbolos & pictogramas
                                   u"\U0001F680-\U0001F6FF"  # transporte & símbolos mapas
                                   u"\U0001F1E0-\U0001F1FF"  # bandeiras (iOS)
                                   u"\U00002500-\U00002BEF"  # caracteres chineses/japoneses coreanos unificados
                                   u"\U00002702-\U000027B0"
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   u"\U0001f926-\U0001f937"
                                   u"\U00010000-\U0010ffff"
                                   u"\u2640-\u2642"
                                   u"\u2600-\u2B55"
                                   u"\u200d"
                                   u"\u23cf"
                                   u"\u23e9"
                                   u"\u231a"
                                   u"\ufe0f"  # dingbats
                                   u"\u3030"
                                   "]+", flags=re.UNICODE)
        text = emoji_pattern.sub(r'', text)

        # Remover caracteres especiais
        text = re.sub(r'[^\w\s]', '', text)

        # Tokenizar o texto em palavras
        tokens = word_tokenize(text)

        # Remover stopwords
        stop_words = set(stopwords.words('english')) # Defina o idioma apropriado
        tokens = [word for word in tokens if word not in stop_words]

        # Lematização (redução das palavras às suas formas base)
        lemmatizer = WordNetLemmatizer() # Utilize o lematizador apropriado para o idioma
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

        # Juntar as palavras de volta em texto
        preprocessed_text = ' '.join(tokens)
    else:
        preprocessed_text = np.nan  # Caso não seja uma string, mantenha o valor como NaN

    return preprocessed_text

# Aplicar a função de pré-processamento à coluna "clean_text"
df['clean_text_2'] = df['text'].apply(preprocess_text)

In [ ]:
# Remover linhas com valores nulos na coluna 'clean_text_2'
df = df.dropna(subset=['clean_text_2'])

In [ ]:
# Converter todos os valores da coluna "clean_text" para strings
df['clean_text_2'] = df['clean_text_2'].astype(str)

In [ ]:
# Concatenar todos os textos em uma única string
all_text = ' '.join(df['clean_text_2'])

In [ ]:
# Exibir o DataFrame resultante
df.head()

# **Modelo LLM - Gemma**

In [ ]:
from transformers import GemmaModel, GemmaTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [ ]:
# Verifique se o GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

In [ ]:
# login huggingface
!huggingface-cli login

In [ ]:
# Base dados
df

In [ ]:
# Importando biblioteca
from sklearn.preprocessing import LabelEncoder

# Codificação dos sentimentos em valores numéricos
encoder = LabelEncoder()
df['sentiment_encoded'] = encoder.fit_transform(df['sentiment'])

# Visualziando dataset
df

In [ ]:
## Vetorizar os textos

# Importando bilioteca
from sklearn.feature_extraction.text import TfidfVectorizer

# Vetorizando
vetorizador = TfidfVectorizer()

# Treinamento
X = vetorizador.fit_transform(df.clean_text_2)

# Visualizando
vetorizador

In [ ]:
import torch
from transformers import GPT2Model, GPT2Tokenizer
from transformers import GemmaModel, GemmaConfig, GemmaTokenizer

In [ ]:
# Verifique se a GPU está disponível
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

In [ ]:
# Supondo que 'dataset' é o seu conjunto de dados
texts = df['clean_text_2'].tolist()

In [ ]:
# Modelo LLM com GPU
model = GPT2Model.from_pretrained("google/gemma-2b").to(device)

In [ ]:
# Carregar o tokenizador e o modelo Gemma
tokenizer = GemmaTokenizer.from_pretrained("google/gemma-2b")

In [ ]:
# Tokenizar e codificar os textos
inputs = tokenizer(texts,
                   return_tensors="pt",
                   padding=True,
                   truncation=True,
                   max_length=512)

In [ ]:
import math

# Configurações de batch e truncamento
max_length = 128  # Define o tamanho máximo dos textos
batch_size = 8    # Define o tamanho do batch

# Calcula o número total de épocas
total_epochs = math.ceil(len(texts) / batch_size)

# Lista para armazenar as representações vetoriais
text_embeddings = []

# Processamento em lotes para reduzir a memória
for epoch in range(total_epochs):
    # Calcula o índice inicial e final do lote atual
    start_index = epoch * batch_size
    end_index = min((epoch + 1) * batch_size, len(texts))

    # Extrai o lote atual de textos
    batch_texts = texts[start_index:end_index]

    # Tokeniza os textos
    inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=max_length).to(device)

    # Obtém a representação vetorial dos textos
    with torch.no_grad():
        outputs = model(**inputs)

    # Usa a última camada oculta como representação vetorial do texto
    batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Movendo para CPU para economizar memória

    # Adiciona as representações vetoriais do lote à lista
    text_embeddings.extend(batch_embeddings)

    # Imprime treinamento
    print(f"Treinamento Modelo {epoch + 1}/{total_epochs} LLM")

# Convertendo para numpy array
text_embeddings = np.array(text_embeddings)

In [ ]:
### Encontrar o valor cluster

# Importando biblioteca
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Calcula a inércia para diferentes números de clusters
inertias = []
max_clusters = 10
for k in range(2, max_clusters + 1):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(text_embeddings)
    inertias.append(kmeans.inertia_)

# Encontra o ponto de cotovelo
diff = np.diff(inertias)
diff_r = diff[1:] / diff[:-1]

# Plotando o gráfico da inércia em função do número de clusters
plt.figure(figsize=(10, 5))
plt.plot(range(2, max_clusters + 1), inertias, marker='o')
plt.xlabel('Número de Clusters')
plt.ylabel('Inércia')
plt.title('Método do Cotovelo')
plt.xticks(np.arange(2, max_clusters + 1, step=1))
plt.grid(False)
plt.show()

# Encontrando o número ideal de clusters usando a regra do cotovelo
optimal_k = np.argmax(diff_r) + 2  # Adiciona 2 porque começamos em k=2
print("Número ideal de clusters (regra do cotovelo):", optimal_k)

In [ ]:
## Modelo kmeans

# Clusterização com K-means usando o número ideal de clusters
kmeans = KMeans(n_clusters=optimal_k, random_state=42)

# Treinamento modelo
kmeans.fit(text_embeddings)

In [ ]:
# Numero cluster
clusters = kmeans.labels_

In [ ]:
# Adicione os clusters ao conjunto de dados
df['cluster'] = clusters

# Visualiznado
df.head()

In [ ]:
# Visualize os resultados
for cluster_num in range(optimal_k):
    print(f"Cluster {cluster_num}:")
    cluster_indices = df[df['cluster'] == cluster_num].index.tolist()
    for idx in cluster_indices:
        print(f"Text: {texts[idx]}")
    print()

In [ ]:
# Imprimir os rótulos dos clusters e o número de clusters únicos
print("Rótulos dos Clusters:", kmeans.labels_)
print("Número de Clusters Únicos:", optimal_k)

In [ ]:
from sklearn.metrics import silhouette_score

# Avaliação da qualidade da clusterização
num_clusters = len(np.unique(kmeans.labels_))
if num_clusters > 1:
    silhouette_avg = silhouette_score(X, kmeans.labels_)
    print("Silhouette Score:", silhouette_avg)
else:
    print("Não é possível calcular o escore de silhueta com apenas um cluster.")

In [ ]:
%%time

# Importando biblioteca
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.decomposition import TruncatedSVD

# Reduzir a dimensionalidade dos vetores de características usando o TruncatedSVD
tsne = TruncatedSVD(n_components=2)
X_tsne_Truncated_SVD = tsne.fit_transform(text_embeddings)

# Reduzir a dimensionalidade dos vetores de características usando o PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(text_embeddings)

# Reduzir a dimensionalidade dos vetores de características usando o t-SNE
tsne = TSNE(n_components=2, perplexity=200)
X_tsne = tsne.fit_transform(text_embeddings)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.lines import Line2D

# Função para criar legendas personalizadas
def create_custom_legend(labels, colors, markers, title):
    custom_legend = []
    for label, color, marker in zip(labels, colors, markers):
        custom_legend.append(Line2D([0], [0], marker=marker, color='w', label=label, markerfacecolor=color, markersize=10))
    return custom_legend

# Lista de cores e marcadores para cada sentimento
colors_sentimentos = {'positivo': 'blue', 'negativo': 'red', 'neutro': 'green'}
markers_sentimentos = {'positivo': 'o', 'negativo': 's', 'neutro': 'D'}

# Plotar os clusters com cores diferentes para cada sentimento usando PCA, t-SNE e TruncatedSVD
plt.figure(figsize=(18, 6))

# Plotar os clusters usando PCA
plt.subplot(1, 3, 1)
sns.scatterplot(x=X_pca[:, 0],
                y=X_pca[:, 1],
                hue=df['cluster'],
                style=df['sentiment'],
                palette='tab10',  # Usando uma paleta padrão com 10 cores diferentes
                markers=['o', 's', 'D'],  # Definindo marcadores padrão
                legend='full')
plt.title('Clusters de Textos com Sentimentos (PCA) - LLM Gemma')
plt.xlabel('Componente 1 (PCA)')
plt.ylabel('Componente 2 (PCA)')
plt.legend(handles=create_custom_legend(legendas_sentimentos.values(), colors_sentimentos.values(), markers_sentimentos.values(), 'Sentimento'))

# Plotar os clusters usando t-SNE
plt.subplot(1, 3, 2)
sns.scatterplot(x=X_tsne[:, 0],
                y=X_tsne[:, 1],
                hue=df['cluster'],
                style=df['sentiment'],
                palette='tab10',
                markers=['o', 's', 'D'],
                legend='full')
plt.title('Clusters de Textos com Sentimentos (t-SNE) - LLM Gemma')
plt.xlabel('Componente 1 (t-SNE)')
plt.ylabel('Componente 2 (t-SNE)')
plt.legend(handles=create_custom_legend(legendas_sentimentos.values(), colors_sentimentos.values(), markers_sentimentos.values(), 'Sentimento'))

# Plotar os clusters usando TruncatedSVD
plt.subplot(1, 3, 3)
sns.scatterplot(x=X_tsne_Truncated_SVD[:, 0],
                y=X_tsne_Truncated_SVD[:, 1],
                hue=df['cluster'],
                style=df['sentiment'],
                palette='tab10',
                markers=['o', 's', 'D'],
                legend='full')
plt.title('Clusters de Textos com Sentimentos (TruncatedSVD) - LLM Gemma')
plt.xlabel('Componente 1 (TruncatedSVD)')
plt.ylabel('Componente 2 (TruncatedSVD)')
plt.legend(handles=create_custom_legend(legendas_sentimentos.values(), colors_sentimentos.values(), markers_sentimentos.values(), 'Sentimento'))

plt.tight_layout()

# Salvar as imagens
plt.savefig('clusters_pca_tsne_truncatedsvd.png')  # Nome do arquivo pode ser alterado conforme necessário

# Visualizar os gráficos
plt.show()


In [ ]:
# Salvando dataset
df.to_csv('dataset_clusterizado_LLM.csv', index=False)

In [ ]:
## Salvando modelo

# Importando biblioteca
import joblib

# Salvar o modelo KMeans em um arquivo
joblib.dump(kmeans, 'modelo_kmeans_LLM.pkl')